In [13]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import platform
plt.style.use('seaborn')
from datetime import datetime
import json

from metric import E1_loss, E2_loss, total_loss
from models import classifier, cnn_model, conv_block, cnn_parallel
from utils import train_model, eval_model, dfDataset, weights_init

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import KFold, train_test_split
from torchsummary import summary

### class, function...

In [14]:
class dfDataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.target = y
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index], self.target[index]
    
def weights_init(m, initializer = nn.init.kaiming_uniform_):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        initializer(m.weight)
        
def train_model(model, train_data, weight, optimizer, loss_func):
    loss_sum = 0
    for i, (x, y) in enumerate(train_data):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        pred = model(x)
        loss = loss_func(pred, y)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
    
    return loss_sum / len(train_data)

def eval_model(model, val_data, loss_func):
    with torch.no_grad():
        loss = 0
        for i, (x, y) in enumerate(val_data):
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss += loss_func(pred, y).item()
    return loss / len(val_data)

class conv_bn(nn.Module):
    def __init__(self, i_f, o_f, fs):
        super(conv_bn, self).__init__()
        self.conv = nn.Conv2d(i_f, o_f, fs)
        self.act = nn.ELU()
        self.bn = nn.BatchNorm2d(o_f)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride= (2, 1))
    def forward(self, x):
        x = self.bn(self.act(self.conv(x)))
        return self.pool(x)
        #return x
    
class conv_block(nn.Module):
    def __init__(self, h_list, input_shape, fs):
        '''
        input_shape : not include batch_size
        '''
        
        super(conv_block, self).__init__()
        self.input_shape = input_shape
        self.fs = fs
        convs = []
        for i in range(len(h_list)):
            if i == 0:
                convs.append(conv_bn(self.input_shape[0], h_list[i], fs))
            else:
                convs.append(conv_bn(h_list[i-1], h_list[i], fs))
        self.convs = nn.Sequential(*convs)
    
    def forward(self, x):
        return self.convs(x)
    
class classifier(nn.Module):
    def __init__(self, h_list, input_size, output_size):
        super(classifier, self).__init__()
        layers = []
        for i in range(len(h_list)):
            if i == 0:
                layers.append(nn.Linear(input_size, h_list[0]))
            else:
                layers.append(nn.Linear(h_list[i-1], h_list[i]))
            layers.append(nn.ELU())
            
        layers.append(nn.Linear(h_list[i], output_size))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)
    
class cnn_model(nn.Module):
    def __init__(self, cnn_block, fc_block):
        super(cnn_model, self).__init__()
        self.cnn = cnn_block
        self.fc = fc_block
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(start_dim = 1)
        return self.fc(x)

def E1_loss(y_pred, y_true):
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2, axis = 1)) / 2e+04

def E2_loss(y_pred, y_true):
    _t, _p = y_true, y_pred
    
    return torch.mean(torch.mean((_t - _p) ** 2 / (_t + 1e-06), axis = 1))

- sequential predict(XY-> V-> M)
- augmentation(noise add)
- channel concat

### Configuration

In [15]:
EPOCH = 100
base_lr = 0.001
now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
save_path = './model/{}'.format(now)
initialize = True
print_summary = True
batch_size = 256
nfold = 10

### load dataset

In [16]:
if platform.system() == 'Windows':
    root_dir = 'D:/datasets/KAERI_dataset/'
else:
    root_dir = '/home/bskim/project/kaeri/KAERI_dataset/'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

train_f = train_f[['Time','S1','S2','S3','S4']].values
train_f = train_f.reshape((-1, 1, 375, 5))#.astype(np.float32)

test_f = test_f[['Time','S1','S2','S3','S4']].values
test_f = test_f.reshape((-1, 1, 375, 5))#.astype(np.float32)
test_f = torch.FloatTensor(test_f)

### Train

In [17]:
def kfold_train(name, feature, target):
    print('{} train...'.format(name))

# make dataset
    train_target = target[list(name)].values#.astype(np.float32)

    # trainx, valx, trainy, valy = train_test_split(train_f, train_target, test_size = 0.2, shuffle = True, random_state = 38)
    fold = KFold(nfold, shuffle = True, random_state= 25)
    loss_per_cv = []
    for i, (train_idx, val_idx) in enumerate(fold.split(feature, y = train_target)):
        print('fold {}'.format(i+1))
        trainx = feature[train_idx]
        valx = feature[val_idx]
        trainy = train_target[train_idx]
        valy = train_target[val_idx]

        train_dataset = dfDataset(trainx.astype(np.float32), trainy)
        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
        val_dataset = dfDataset(valx.astype(np.float32), valy)
        val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

        n_features = feature.shape[-1]
        #fc = classifier([128, 64, 32, 16], input_size = 256*2*5, output_size = len(name))
        #conv = conv_block([32, 64, 128, 256, 256, 256, 256], [1, 375, 5], (3, 1))
        conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, n_features], (3, 1))
        fc = classifier([128, 64, 32, 16], input_size = 512*3*n_features, output_size = len(name))
        # define model
        model = cnn_model(conv, fc)
        optimizer = torch.optim.Adam(model.parameters(), lr = base_lr)

        if name == 'XY':
            criterion = E1_loss
        else:
            criterion = E2_loss

        model = model.cuda()
        if initialize:
            model.apply(weights_init)

        curr_loss = 1e+7
        os.makedirs(save_path) if not os.path.exists(save_path) else None
        #train
        for ep in range(1, EPOCH + 1):
            model.train()
            loss = train_model(model, train_loader, criterion, optimizer, criterion)
            model.eval()
            val_loss =eval_model(model, val_loader, criterion)
            if curr_loss > val_loss:
                print('[{}] : train loss {:4f}, val loss drop {:.4f} to {:.4f}'.format(ep, np.mean(loss), curr_loss, val_loss))
                curr_loss = val_loss
                torch.save(model.state_dict(), os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i+1)))
        loss_per_cv.append(curr_loss)
    return loss_per_cv           

In [18]:
# train XY
loss_xy = kfold_train('XY',train_f, train_t)

XY train...
fold 1
[1] : train loss 2.560765, val loss drop 10000000.0000 to 1.9493
[2] : train loss 1.150106, val loss drop 1.9493 to 0.6211
[3] : train loss 0.426928, val loss drop 0.6211 to 0.1966
[4] : train loss 0.105472, val loss drop 0.1966 to 0.0751
[5] : train loss 0.033847, val loss drop 0.0751 to 0.0472
[6] : train loss 0.023009, val loss drop 0.0472 to 0.0175
[7] : train loss 0.011714, val loss drop 0.0175 to 0.0105
[8] : train loss 0.007423, val loss drop 0.0105 to 0.0086
[9] : train loss 0.004749, val loss drop 0.0086 to 0.0052
[11] : train loss 0.002415, val loss drop 0.0052 to 0.0033
[12] : train loss 0.002308, val loss drop 0.0033 to 0.0032
[13] : train loss 0.002254, val loss drop 0.0032 to 0.0023
[19] : train loss 0.001162, val loss drop 0.0023 to 0.0022
[20] : train loss 0.000986, val loss drop 0.0022 to 0.0021
[28] : train loss 0.000915, val loss drop 0.0021 to 0.0012
[58] : train loss 0.000484, val loss drop 0.0012 to 0.0010
[61] : train loss 0.000456, val loss dr

In [19]:
add_feature = train_t[['X','Y']].values.reshape((2800, 1, 1, 2))
add_feature = np.repeat(add_feature, 375, axis = 2)
trainX = np.concatenate((train_f, add_feature), axis = -1)

# train V using XY
loss_v = kfold_train('V',trainX, train_t)

V train...
fold 1
[1] : train loss 15.645872, val loss drop 10000000.0000 to 5.9779
[2] : train loss 2.245281, val loss drop 5.9779 to 2.1940
[3] : train loss 0.564295, val loss drop 2.1940 to 0.1580
[4] : train loss 0.145717, val loss drop 0.1580 to 0.1429
[5] : train loss 0.061145, val loss drop 0.1429 to 0.0397
[6] : train loss 0.032252, val loss drop 0.0397 to 0.0290
[7] : train loss 0.020974, val loss drop 0.0290 to 0.0174
[10] : train loss 0.012447, val loss drop 0.0174 to 0.0142
[11] : train loss 0.011588, val loss drop 0.0142 to 0.0138
[14] : train loss 0.011766, val loss drop 0.0138 to 0.0108
[22] : train loss 0.007213, val loss drop 0.0108 to 0.0087
[23] : train loss 0.003726, val loss drop 0.0087 to 0.0065
[26] : train loss 0.003954, val loss drop 0.0065 to 0.0061
[32] : train loss 0.004627, val loss drop 0.0061 to 0.0058
[33] : train loss 0.005190, val loss drop 0.0058 to 0.0045
[47] : train loss 0.002061, val loss drop 0.0045 to 0.0039
[58] : train loss 0.006324, val loss 

In [20]:
add_feature = train_t[['V']].values.reshape((2800, 1, 1, 1))
add_feature = np.repeat(add_feature, 375, axis = 2)
trainX = np.concatenate((trainX, add_feature), axis = -1)

# train V using XY
loss_m = kfold_train('M',trainX, train_t)

M train...
fold 1
[1] : train loss 36.495263, val loss drop 10000000.0000 to 7.0307
[3] : train loss 1.723455, val loss drop 7.0307 to 3.3378
[5] : train loss 0.679273, val loss drop 3.3378 to 1.9550
[6] : train loss 0.808075, val loss drop 1.9550 to 0.3929
[9] : train loss 0.278975, val loss drop 0.3929 to 0.2885
[14] : train loss 0.344460, val loss drop 0.2885 to 0.1133
[18] : train loss 0.244292, val loss drop 0.1133 to 0.1099
[19] : train loss 0.223345, val loss drop 0.1099 to 0.0907
[22] : train loss 0.373950, val loss drop 0.0907 to 0.0860
[25] : train loss 0.127380, val loss drop 0.0860 to 0.0714
[38] : train loss 0.179388, val loss drop 0.0714 to 0.0523
[59] : train loss 0.103606, val loss drop 0.0523 to 0.0477
[66] : train loss 0.464927, val loss drop 0.0477 to 0.0442
[92] : train loss 0.044855, val loss drop 0.0442 to 0.0409
[95] : train loss 0.110060, val loss drop 0.0409 to 0.0379
[98] : train loss 0.080377, val loss drop 0.0379 to 0.0317
[100] : train loss 0.085395, val lo

In [21]:
loss_per_model = {'xy':loss_xy, 'v':loss_v, 'm':loss_m}

In [22]:
with open(os.path.join(save_path, 'loss_info.json'), 'w') as f:
    for k in loss_per_model:
        loss_per_model[k] = np.mean(loss_per_model[k])
    f.write(json.dumps(loss_per_model))

## test

In [23]:
# predict XY
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
name = 'XY'
n_features = test_f.size()[-1]
conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, n_features], (3, 1))
fc = classifier([128, 64, 32, 16], input_size = 512*3*n_features, output_size = len(name))
# define model
model = cnn_model(conv, fc)
pred_array = []
for i in range(1, nfold + 1):
    model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
    model = model.cuda()

    with torch.no_grad():
        predict = model(test_f.cuda())
    pred_array.append(predict.detach().cpu().numpy())
result = np.mean(pred_array, axis = 0)
submission[list(name)] = result

In [24]:
n_samples = test_f.shape[0]
add_feature_t = result.reshape((n_samples, 1, 1, len(name)))
add_feature_t = np.repeat(add_feature_t, 375, axis = 2)

add_feature_t = torch.FloatTensor(add_feature_t)

test_f_add = torch.cat([test_f, add_feature_t], dim = -1)

In [25]:
# predict V
name = 'V'
n_features = test_f_add.size()[-1]
conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, n_features], (3, 1))
fc = classifier([128, 64, 32, 16], input_size = 512*3*n_features, output_size = len(name))
# define model
model = cnn_model(conv, fc)
pred_array = []
for i in range(1, nfold + 1):
    model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
    model = model.cuda()

    with torch.no_grad():
        predict = model(test_f_add.cuda())
    pred_array.append(predict.detach().cpu().numpy())
result = np.mean(pred_array, axis = 0)
submission[list(name)] = result

In [26]:
n_samples = test_f_add.shape[0]
add_feature_t = result.reshape((n_samples, 1, 1, len(name)))
add_feature_t = np.repeat(add_feature_t, 375, axis = 2)

add_feature_t = torch.FloatTensor(add_feature_t)

test_f_add = torch.cat([test_f_add, add_feature_t], dim = -1)

In [27]:
# predict M
name = 'M'
n_features = test_f_add.size()[-1]
conv = conv_block([16, 32, 64, 128, 256, 512], [1, 375, n_features], (3, 1))
fc = classifier([128, 64, 32, 16], input_size = 512*3*n_features, output_size = len(name))
# define model
model = cnn_model(conv, fc)
pred_array = []
for i in range(1, nfold + 1):
    model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
    model = model.cuda()

    with torch.no_grad():
        predict = model(test_f_add.cuda())
    pred_array.append(predict.detach().cpu().numpy())
result = np.mean(pred_array, axis = 0)
submission[list(name)] = result

In [28]:
submission.head()

,id,X,Y,M,V
0,2800,-260.121490,-42.052998,113.206131,0.450548
1,2801,315.630432,-280.351257,89.356171,0.481469
2,2802,-237.003372,129.876831,29.687794,0.395682
3,2803,159.593109,274.795746,27.364025,0.409057
4,2804,-169.503418,187.204330,132.531036,0.452389


In [29]:
submission.to_csv(os.path.join(save_path, 'submit.csv'), index = False)